In [1]:
import psycopg2
import torch
import pickle
import sqlite3
import pandas as pd
import numpy as np

In [3]:
def connect():
    return psycopg2.connect(database="test_insert",
                        host="localhost",
                        user="maxj",
                        password="",
                        port="5432")


In [17]:
def reset_qq():
    conn = connect()
    c = conn.cursor()
    c.execute("drop table if exists qq;")
    c.execute("""
    CREATE TABLE qq (
      id BIGINT PRIMARY KEY,
      payload JSONB NOT NULL,
      customer_id BIGINT,
      deliver_after BIGINT,
      attempts INT,
      expires_after BIGINT
    );
    CREATE INDEX qq_deliver ON qq (deliver_after);
    """)
    conn.commit()
    c.close()
    conn.close()

reset_qq()

In [18]:
import time

payload = '''
{
  "user_id": 12345,
  "username": "example_user",
  "email": "user@example.com",
  "age": 30,
  "is_active": true,
  "preferences": {
    "theme": "dark",
    "notifications": true,
    "language": "en"
  },
  "posts": [
    {
      "post_id": 1,
      "title": "First Post",
      "content": "This is the content of the first post."
    },
    {
      "post_id": 2,
      "title": "Second Post",
      "content": "This is the content of the second post."
    },
    {
      "post_id": 3,
      "title": "Third Post",
      "content": "This is the content of the third post."
    }
  ]
}
'''

def bench(n_iters=1000000):
    conn = connect()
    conn.set_session(autocommit=True)
    for i in range(n_iters):
        c = conn.cursor()
        now = time.time()
        deliver_after = round(now * 1000)
        expires_after = round(now * 1000) + 1000000
        c.execute("INSERT INTO qq VALUES (%s, %s, %s, %s, %s, %s)", (i, payload, 1, deliver_after, 0, expires_after))
        c.close()
    conn.close()
    return n_iters

start = time.time()
n_iters = bench(100000)
end = time.time()
duration = end - start
rate = n_iters / duration
rate

8484.120188897765